# Parameter Efficient Fine Tuning: Adapter Layers on Financial Sentiment Analysis

Read a more detailed overview on fine-tuning using adapter layers: <https://arxiv.org/abs/2303.15647> [^1]

Also, you can read PEFT on Huggingface: <https://huggingface.co/blog/peft>

[^1]: Lialin, Vladislav, Vijeta Deshpande, and Anna Rumshisky. "Scaling down to scale up: A guide to parameter-efficient fine-tuning." arXiv preprint arXiv:2303.15647 (2023).

In [1]:
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_model,
    PeftType,
    PromptTuningConfig,
)

import evaluate
from transformers import BertTokenizer,BertForSequenceClassification, BertConfig, get_linear_schedule_with_warmup
from tqdm import tqdm

In this notebook, we will use Indian Financial News dataset (<https://huggingface.co/datasets/kdave/Indian_Financial_News>) to improve the model's performance of analysing news sources from different geographies. In our previous experiments, we observed that 0.88 accuracy of FinBERT model performance reduced to 0.57 in three category classification task. The dataset includes equal number of samples (n=8987) from each tone category "Positive", "Negative" and "Neutral". Fine-tuning using a real-word, different country-sourced dataset is important for fairness to improve model's performance on different technical jargon, currency, and other implicit bias sources. 

In [2]:
from datasets import load_dataset

ds = load_dataset("kdave/Indian_Financial_News", split="train")

In [3]:
ds = ds.train_test_split(test_size=0.2)

In [4]:
# Remove URL and Content columns
ds = ds.remove_columns(["URL", "Content"])

# Sentiment column encode labels "Positive" as 2, "Neutral" as 1, and "Negative" as 0
ds = ds.map(
    lambda x: {"Sentiment": 2 if x["Sentiment"] == "Positive" else 1 if x["Sentiment"] == "Neutral" else 0}
)

Map:   0%|          | 0/21568 [00:00<?, ? examples/s]

Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

In [5]:
ds["train"]

Dataset({
    features: ['Summary', 'Sentiment'],
    num_rows: 21568
})

In [6]:
ds["test"]

Dataset({
    features: ['Summary', 'Sentiment'],
    num_rows: 5393
})

### Model and Data Tokenization

In [7]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
config = BertConfig.from_pretrained('yiyanghkust/finbert-tone')

/Users/asabuncuoglu/Documents/faid/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
batch_size = 32
task = "mrpc"
peft_type = PeftType.PROMPT_TUNING
num_epochs = 20

In [23]:
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples["Summary"], max_length=None, truncation=True, padding="max_length")
    return outputs

tokenized_datasets = ds.map(
    tokenize_function,
    batched=True,
    remove_columns=["Summary"],
)

tokenized_datasets = tokenized_datasets.rename_column("Sentiment", "labels")

def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)

Map:   0%|          | 0/21568 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

### Add adapter layers

In [24]:
peft_config = PromptTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=10)
lr = 1e-3

In [25]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 9,987 || all params: 109,764,102 || trainable%: 0.0091


In [26]:
len(train_dataloader)

674

In [27]:
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForSequenceClassification(
  (base_model): PeftModelForSequenceClassification(
    (base_model): PeftModelForSequenceClassification(
      (base_model): BertForSequenceClassification(
        (bert): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30873, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0-11): 12 x BertLayer(
                (attention): BertAttention(
                  (self): BertSdpaSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_features=768, out_features=768, bias=True)
                    (value): Linear(in_features=768, out_f

In [30]:
metric = evaluate.load("accuracy")

for epoch in range(1):
#for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

100%|██████████| 169/169 [02:54<00:00,  1.03s/it]

epoch 0: {'accuracy': 0.5839050621175598}


In [31]:
model.save_pretrained("./model/finbert-peft-prompt-tuning", from_pt=True)

## Use the Model

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer

peft_model_id = "./model/finbert-peft-prompt-tuning"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
inference_model = PeftModel.from_pretrained(inference_model, peft_model_id)

inference_model.to(device)
inference_model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = inference_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )

eval_metric = metric.compute()
print(eval_metric)